In [1]:
import psi4
import numpy as np

speed_of_light = 299792458
hartree_to_joule = 4.35974e-18
bohr_to_meter = 5.2918e-11
amu_to_kg = 1.66054e-27
unit_conversion_factor = hartree_to_joule/(bohr_to_meter)**2/amu_to_kg

In [2]:
h2o = psi4.geometry("""
0 1
    O            0.000000000000    -0.000000000000    -0.064730226948
    H            0.748952681885     0.000000000000     0.513657837538
    H           -0.748952681885    -0.000000000000     0.513657837538
""")

basis = "cc-pvdz"

psi4.set_options({'basis':basis,
                 'reference':'rhf',
                 'scf_type':'pk'})

psi4.optimize('hf')
H = psi4.hessian('hf')

Optimizer: Optimization complete!


In [3]:
N = h2o.natom()
coordinates = np.zeros((N,3))
cm_coordinates = np.zeros((N,3))
M_vec = []
R_cm = np.zeros(3)
I = np.zeros((3,3))
I0 = 0

In [4]:
for i in range(N):
    coordinates[i,:] = np.array([h2o.x(i),h2o.y(i),h2o.z(i)])
    R_cm += h2o.mass(i)*coordinates[i,:]
    lst = [h2o.mass(i),h2o.mass(i),h2o.mass(i)]
    M_vec += lst
M_vec = np.array(M_vec)
R_cm = R_cm*3/np.sum(M_vec)
cm_coordinates = coordinates - R_cm
for i in range(N):
    I += h2o.mass(i)*np.outer(cm_coordinates[i,:],cm_coordinates[i,:])
    I0 += h2o.mass(i)*np.dot(cm_coordinates[i,:],cm_coordinates[i,:])
I = I0*np.eye(3) - I

In [5]:
Ieigval,X = np.linalg.eig(I)

In [6]:
# Translational basis
S1 = np.zeros((N,3))
S2 = np.zeros((N,3))
S3 = np.zeros((N,3))
# Rotational basis
S4 = np.zeros((N,3))
S5 = np.zeros((N,3))
S6 = np.zeros((N,3))

In [7]:
for i in range(N):
    S1[i,:] = X[:,0].T
    S2[i,:] = X[:,1].T
    S3[i,:] = X[:,2].T
    S4[i,:] = np.cross(cm_coordinates[i,:],X[:,0].T)
    S5[i,:] = np.cross(cm_coordinates[i,:],X[:,1].T)
    S6[i,:] = np.cross(cm_coordinates[i,:],X[:,2].T)

S1 = np.sqrt(M_vec) * S1.flatten()
S2 = np.sqrt(M_vec) * S2.flatten()
S3 = np.sqrt(M_vec) * S3.flatten()
S4 = np.sqrt(M_vec) * S4.flatten()
S5 = np.sqrt(M_vec) * S5.flatten()
S6 = np.sqrt(M_vec) * S6.flatten()

In [8]:
S = np.zeros((3*N,3*N+6))

In [9]:
S[:,0] = S1.T
S[:,1] = S2.T
S[:,2] = S3.T
S[:,3] = S4.T
S[:,4] = S5.T
S[:,5] = S6.T
S[:,6:] = np.eye(3*N)

In [10]:
D,tmp = np.linalg.qr(S)

In [11]:
H_card = np.array(H)
H_weight = (np.outer(M_vec,M_vec))**(-1/2)*H_card
H_ext_int = D.T @ H_weight @ D
H_int = H_ext_int[6:,6:]

In [12]:
normal_mode = np.linalg.eigvals(H_int)

In [13]:
all_mode = np.linalg.eigvals(H_weight)

In [14]:
normal_mode 

array([0.67143628, 0.64036964, 0.11929156])

In [15]:
all_mode

array([ 6.71436277e-01,  6.40369644e-01,  1.19291557e-01,  1.17525098e-05,
       -3.29762517e-05,  3.54448810e-05, -7.41383781e-13, -7.32960656e-13,
       -7.30265282e-13])